In [1]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
# Allows plots to appear directly in the notebook.
%matplotlib inline
# For dealing with some Accented characters (in Irish Place names)
import unidecode
# Date/time functionality
import datetime
import time
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
# look at some z-scores for inspecting outliers.
from scipy import stats
import seaborn as sb
# lookup lat/long and convert lat/long to national grid references.
import geopy
import pyproj

from patsy import dmatrices
from sklearn import metrics

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
#from sklearn.tree import export_text

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

import graphviz
from graphviz import Source
#to read all CSV files in a folder
import os
import glob

Reading data in chunks and adding dataframes to a list
        
        *Note: Dont need to run below cell if chunks already created, call get_chunks function to create chunks list

In [2]:
chunk_size = 10000000
batch_no=1
chunk_trips_list=[]
for chunk in pd.read_csv('~/tmp/data/rt_trips_DB_2018.txt',sep=';',chunksize=chunk_size):
    chunk_trips_list.append(chunk)
    chunk.to_csv('Chunks/trips/'+str(batch_no)+'chunk_trips'+'.csv',index=False)
    batch_no+=1
    print(chunk.shape)

(2182637, 16)


Reading Data from multiple files in a folder to a list of data frames

In [2]:
def get_chunks(location):
    cwd=os.getcwd()
    path = cwd+location
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    for filename in chunk_folder:
        # read the csv file
        df_chunk = pd.read_csv(filename)
        chunk_list.append(df_chunk)
        print('Location:', filename)
    return chunk_list

In [4]:
chunk_trips_list=get_chunks('/Chunks/trips')

('Location:', '/home/team8/notebook/Pelin/Chunks/trips/1chunk_trips.csv')


Combining chunks to 1 big dataframe

In [5]:
df_trips=pd.concat(chunk_trips_list, axis=0)

In [6]:
df_trips.shape

(2182637, 16)

In [7]:
df_trips.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


In [8]:
df_trips.tail(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
2182632,DB,14-MAY-18 00:00:00,6765849,123,123_36,2,61560,57840,61365.0,57859.0,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",3216350,"
2182633,DB,14-MAY-18 00:00:00,6765469,75,75_17,1,53416,48600,NaN,48823.0,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",2865284,"
2182634,DB,14-MAY-18 00:00:00,6765486,33D,33D_62,2,29460,26400,29904.0,NaN,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",3077688,"
2182635,DB,14-MAY-18 00:00:00,6764987,70,70_60,1,65277,60600,66341.0,NaN,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",3208841,"
2182636,DB,14-MAY-18 00:00:00,6765012,27,27_19,1,47722,41700,47508.0,41642.0,BasDef,NaN,NaN,NaN,26-JUN-18 09:13:13,",2960092,"


In [9]:
for df in chunk_trips_list:
    print(df.nunique())
    print ("="*66)

DATASOURCE              1
DAYOFSERVICE          360
TRIPID             658964
LINEID                130
ROUTEID               588
DIRECTION               2
PLANNEDTIME_ARR     64461
PLANNEDTIME_DEP       791
ACTUALTIME_ARR      68122
ACTUALTIME_DEP      66771
BASIN                   1
TENDERLOT               0
SUPPRESSED              1
JUSTIFICATIONID      3526
LASTUPDATE            360
NOTE                46690
dtype: int64


There is 588 route, 130 Line in this dataset. 

Also this dataset contains information related to 658964 trip whereas, information related to approximately 826029 trip is available at leavetimes dataset(number 826029 might include double counted tripIDs due to summing up uniques in all chunks) 

In [11]:
df_trips.columns

Index([u'DATASOURCE', u'DAYOFSERVICE', u'TRIPID', u'LINEID', u'ROUTEID',
       u'DIRECTION', u'PLANNEDTIME_ARR', u'PLANNEDTIME_DEP', u'ACTUALTIME_ARR',
       u'ACTUALTIME_DEP', u'BASIN', u'TENDERLOT', u'SUPPRESSED',
       u'JUSTIFICATIONID', u'LASTUPDATE', u'NOTE'],
      dtype='object')

Dropping empty and constant columns from all chunks in dataset.

In [12]:
for df in chunk_trips_list:
    del df['DATASOURCE']
    del df['BASIN']
    del df['TENDERLOT']
    del df['SUPPRESSED']

In [13]:
# Clean the column names to remove white space after the name or in the name
for df in chunk_trips_list:
    df.columns = df.columns.str.replace(' ', '')

In [3]:
def update_csv_chunks(adress,name, c_list):
    cwd=os.getcwd()
    path = cwd+adress
    batch_no=1
    for chunk in c_list:
        chunk.to_csv(path+str(batch_no)+name+'.csv',index=False)
        batch_no+=1
        print(chunk.shape)

In [15]:
update_csv_chunks('/Chunks/trips/','chunk_trips', chunk_trips_list)

(2182637, 12)


#### ========================================================================================================

In [5]:
chunk_trips_list=get_chunks('/Chunks/trips')

('Location:', '/home/team8/notebook/Pelin/Chunks/trips/1chunk_trips.csv')


In [6]:
df_trips=pd.concat(chunk_trips_list, axis=0)

Creating PK feature columns to merge with other DB datasets

In [7]:
for df in chunk_trips_list:
    df['PK_4_TRIPS']=df['DAYOFSERVICE'].astype('str')+df['TRIPID'].astype('str')

In [8]:
#updating bigdataframe with newly derived features chunks
df_trips=pd.concat(chunk_trips_list, axis=0)

In [9]:
df_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JUSTIFICATIONID,LASTUPDATE,NOTE,PK_4_TRIPS
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,28-FEB-18 12:05:11,",2967409,",07-FEB-18 00:00:006253783
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,28-FEB-18 12:05:11,",2580260,",07-FEB-18 00:00:006262138
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,28-FEB-18 12:05:11,",2448968,",07-FEB-18 00:00:006254942
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,28-FEB-18 12:05:11,",3094242,",07-FEB-18 00:00:006259460
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,28-FEB-18 12:05:11,",2526331,",07-FEB-18 00:00:006253175


In [11]:
update_csv_chunks('/Chunks/trips/','chunk_trips', chunk_trips_list)

(2182637, 13)


#### ========================================================================================================

Converting features into appropriate data types

In [4]:
chunk_trips_list=get_chunks('/Chunks/trips')

('Location:', '/home/team8/notebook/Pelin/Chunks/trips/1chunk_trips.csv')


In [5]:
df_trips=pd.concat(chunk_trips_list, axis=0)

In [7]:
df_trips.dtypes

DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
PK_4_TRIPS          object
dtype: object

In [8]:
for df in chunk_trips_list:
    df['DAYOFSERVICE']=df['DAYOFSERVICE'].astype('str')
    df['LASTUPDATE']=df['LASTUPDATE'].astype('str')

In [9]:
df_trips=pd.concat(chunk_trips_list, axis=0)

In [10]:
df_trips.dtypes

DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
PK_4_TRIPS          object
dtype: object

In [11]:
df_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JUSTIFICATIONID,LASTUPDATE,NOTE,PK_4_TRIPS
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,28-FEB-18 12:05:11,",2967409,",07-FEB-18 00:00:006253783
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,28-FEB-18 12:05:11,",2580260,",07-FEB-18 00:00:006262138
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,28-FEB-18 12:05:11,",2448968,",07-FEB-18 00:00:006254942
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,28-FEB-18 12:05:11,",3094242,",07-FEB-18 00:00:006259460
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,28-FEB-18 12:05:11,",2526331,",07-FEB-18 00:00:006253175


In [20]:
for df in chunk_trips_list:
    print("NaN values at ACTUALTIME_DEP column :", df['ACTUALTIME_DEP'].isna().sum())
    print("NaN values across dataset :", df.isna().sum().sum())

('NaN values at ACTUALTIME_DEP column :', 164551)
('NaN values across dataset :', 2480065)


This doesn't contain actual arrival or departure times if they are cancled which appear to be NaN in dataset. 
    
    *therefore inorder to be able to cast this column as int and do artihmetic operations;
    *we are filling NaN values at ACTUALTIME_DEP and ACTUALTIME_ARR with 0

In [23]:
for df in chunk_trips_list:
    df['ACTUALTIME_DEP']=df['ACTUALTIME_DEP'].fillna(0)
    df['ACTUALTIME_ARR']=df['ACTUALTIME_ARR'].fillna(0)

In [24]:
for df in chunk_trips_list:
    print("NaN values at ACTUALTIME_DEP column :", df['ACTUALTIME_DEP'].isna().sum())
    print("NaN values at ACTUALTIME_ARR column :", df['ACTUALTIME_ARR'].isna().sum())
    print("NaN values across dataset :", df.isna().sum().sum())

('NaN values at ACTUALTIME_DEP column :', 0)
('NaN values at ACTUALTIME_ARR column :', 0)
('NaN values across dataset :', 2178307)


In [25]:
def trips_list_dtype_converter(df):
    df['DAYOFSERVICE'] =  pd.to_datetime(df['DAYOFSERVICE'], format="%d-%b-%y %H:%M:%S")
    df['TRIPID']=df['TRIPID'].astype('category')
    df['LINEID']=df['LINEID'].astype('category')
    df['ROUTEID']=df['ROUTEID'].astype('category')
    df['DIRECTION']=df['DIRECTION'].astype('category')
    df['ACTUALTIME_ARR']=df['ACTUALTIME_ARR'].astype('int64')
    df['ACTUALTIME_DEP']=df['ACTUALTIME_DEP'].astype('int64')
    df['JUSTIFICATIONID']=df['JUSTIFICATIONID'].astype('str')
    df['LASTUPDATE'] =  pd.to_datetime(df['LASTUPDATE'], format="%d-%b-%y %H:%M:%S")
    df['NOTE']=df['NOTE'].astype('str')
    df['PK_4_TRIPS']=df['PK_4_TRIPS'].astype('str')
    print(df.dtypes)
    return df

In [26]:
chunk_trips_list=map(trips_list_dtype_converter,chunk_trips_list)

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR             int64
PLANNEDTIME_DEP             int64
ACTUALTIME_ARR              int64
ACTUALTIME_DEP              int64
JUSTIFICATIONID            object
LASTUPDATE         datetime64[ns]
NOTE                       object
PK_4_TRIPS                 object
dtype: object


In [27]:
df_trips=pd.concat(chunk_trips_list, axis=0)

In [28]:
update_csv_chunks('/Chunks/trips/','chunk_trips', chunk_trips_list)

(2182637, 13)
